In [1]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

### 이미지 로딩

In [2]:
IMAGE_SIZE = (300, 300)

IMAGE_PATH = "./plant-pathology-2020-fgvc7/images/"
TEST_PATH = "./plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "./plant-pathology-2020-fgvc7/train.csv"
#SUB_PATH = "./plant-pathology-2020-fgvc7/sample_submission.csv"

test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)

    image = cv2.resize(image, IMAGE_SIZE)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

images = test_data["image_id"].apply(load_image)



In [3]:
test_images = np.stack(images)

In [35]:
kfold = 5
df_list = []
for i in range(kfold):
    model = load_model(f"./model/train13/kfold{i}/best.hdf5")
    result = model.predict(test_images)
    result_df = pd.DataFrame(result, columns=["healthy", "multiple_diseases", "rust", "scab"])
    df_list.append(result_df)

57/57 [==============================] - 5s 75ms/step


In [5]:
# 첫 번째 DataFrame을 복사하여 평균을 저장할 DataFrame 생성
result_df = df_list[0].copy()

# 나머지 DataFrame의 셀 값을 더하고 개수를 세어 평균 계산
for df in df_list[1:]:
    result_df += df

result_df /= len(df_list)

# 결과 출력
result_df = pd.concat([test_data, result_df], axis=1)


In [6]:
result_df.to_csv("./result.csv", sep=',', index=False)